<a href="https://colab.research.google.com/github/HeMANSC/PandaSQL/blob/main/pandasql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PandaSQL Project by Hemant Singh

In [ ]:
!pip install pandas pandasql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26772 sha256=b80ff1faf96ab95c8f82af52cde2f2c99870986397588c9b0ad8f0bfe4d8c4c9
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [33]:
import pandas as pd
import sqlite3
from datetime import datetime

In [34]:
df = pd.read_csv('/content/people-1000.csv')
df.head()

,Index,User Id,First Name,Last Name,Sex,Email,Phone,Date of birth,Job Title
0,1,8717bbf45cCDbEe,Shelia,Mahoney,Male,pwarner@example.org,857.139.8239,2014-01-27,Probation officer
1,2,3d5AD30A4cD38ed,Jo,Rivers,Female,fergusonkatherine@example.net,+1-950-759-8687,1931-07-26,Dancer
2,3,810Ce0F276Badec,Sheryl,Lowery,Female,fhoward@example.org,(599)782-0605,2013-11-25,Copy
3,4,BF2a889C00f0cE1,Whitney,Hooper,Male,zjohnston@example.com,+1-939-130-6258,2012-11-17,Counselling psychologist
4,5,9afFEafAe1CBBB9,Lindsey,Rice,Female,elin@example.net,(390)417-1635x3010,1923-04-15,Biomedical engineer


In [35]:
# Calculate Age in the dfFrame
current_year = datetime.now().year
df['Date of birth'] = pd.to_datetime(df['Date of birth'], errors='coerce')
df['Age'] = current_year - df['Date of birth'].dt.year

In [37]:
# Create an SQLite dfbase
conn = sqlite3.connect(':memory:')
df.to_sql('people', conn, index=False, if_exists='replace')

# Add the Age column to the SQLite table
df[['Index', 'Age']].to_sql('temp_age', conn, index=False, if_exists='replace')
cursor = conn.cursor()
cursor.execute("""
    UPDATE people
    SET Age = (
        SELECT Age FROM temp_age WHERE temp_age."Index" = people."Index"
    )
""")
conn.commit()

In [38]:
# Query to find the youngest individual
query_youngest = """
    SELECT
        "First Name", "Last Name", Age, "Date of birth"
    FROM people
    ORDER BY Age ASC
    LIMIT 1
"""
youngest_individual = pd.read_sql_query(query_youngest, conn)

# Query to find the oldest individual
query_oldest = """
    SELECT
        "First Name", "Last Name", Age, "Date of birth"
    FROM people
    ORDER BY Age DESC
    LIMIT 1
"""
oldest_individual = pd.read_sql_query(query_oldest, conn)

# Display results
print("Youngest Individual:")
print(youngest_individual)
print("\nOldest Individual:")
print(oldest_individual)


Youngest Individual:
  First Name Last Name  Age        Date of birth
0      Malik   Summers    3  2022-03-28 00:00:00

Oldest Individual:
  First Name Last Name  Age        Date of birth
0    Yesenia   Harding  119  1906-10-28 00:00:00


In [41]:
# 2. Gender Distribution
query_gender_distribution = """
    SELECT Sex, COUNT(*) AS Count
    FROM people
    GROUP BY Sex
"""
gender_distribution = pd.read_sql_query(query_gender_distribution, conn)
print("Gender Distribution:")
print(gender_distribution)

Gender Distribution:
      Sex  Count
0  Female    494
1    Male    506


In [43]:
# 3. Job Analysis
# a) Most common job titles
query_common_jobs = """
    SELECT "Job Title", COUNT(*) AS Count
    FROM people
    GROUP BY "Job Title"
    ORDER BY Count DESC
    LIMIT 1000
"""
common_jobs = pd.read_sql_query(query_common_jobs, conn)
print("\nMost Common Job Titles:")
print(common_jobs)

# b) Gender-specific trends in professions
query_gender_jobs = """
    SELECT Sex, "Job Title", COUNT(*) AS Count
    FROM people
    GROUP BY Sex, "Job Title"
    ORDER BY Sex, Count DESC
    LIMIT 1000
"""
gender_jobs = pd.read_sql_query(query_gender_jobs, conn)
print("\nGender-Specific Job Trends:")
print(gender_jobs)



Most Common Job Titles:
                        Job Title  Count
0                  Phytotherapist      7
1                Paediatric nurse      7
2             Production engineer      6
3            Nurse, mental health      6
4                    Nurse, adult      6
..                            ...    ...
514           Administrator, arts      1
515  Administrator, Civil Service      1
516                 Acupuncturist      1
517                         Actor      1
518         Accountant, chartered      1

[519 rows x 2 columns]

Gender-Specific Job Trends:
        Sex                        Job Title  Count
0    Female                   Phytotherapist      5
1    Female                   Prison officer      4
2    Female                Personnel officer      4
3    Female                 Paediatric nurse      4
4    Female                     Web designer      3
..      ...                              ...    ...
702    Male                    Administrator      1
703    Male   

In [44]:
# 4. Contact Information
# a) Extract unique email domains
query_email_domains = """
    SELECT SUBSTR(Email, INSTR(Email, '@') + 1) AS Domain, COUNT(*) AS Count
    FROM people
    GROUP BY Domain
    ORDER BY Count DESC
"""
email_domains = pd.read_sql_query(query_email_domains, conn)
print("\nUnique Email Domains:")
print(email_domains)

# b) Check for duplicate phone numbers
query_duplicate_phones = """
    SELECT Phone, COUNT(*) AS Count
    FROM people
    GROUP BY Phone
    HAVING Count > 1
"""
duplicate_phones = pd.read_sql_query(query_duplicate_phones, conn)
print("\nDuplicate Phone Numbers:")
print(duplicate_phones)



Unique Email Domains:
        Domain  Count
0  example.org    341
1  example.com    339
2  example.net    320

Duplicate Phone Numbers:
Empty DataFrame
Columns: [Phone, Count]
Index: []
